In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import time
from scipy.stats import genextreme as gev
import openturns as ot
import openturns.viewer as viewer
import sympy

In [ ]:
marathon_results = "Results_manchester_marathon_2022.xlsx" 

In [ ]:
df = pd.read_excel (marathon_results,index_col='Chiptime', parse_dates=True)
#df = pd.read_excel (marathon_results)

In [ ]:
print(df.columns)

In [ ]:
df

In [ ]:
df.drop(['Bib', 'Name','Team / Club', 'Avg speed', 'Avg pace', 'Status', 'Split - 10K - Duration', 'Split - 10K - Cumulative time',
       'Split - 10K - Rank', 'Split - Halfway - Duration',
       'Split - Halfway - Cumulative time', 'Split - Halfway - Rank',
       'Split - 30K - Duration', 'Split - 30K - Cumulative time',
       'Split - 30K - Rank', 'Split - Finish - Duration',
       'Split - Finish - Cumulative time'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
#data = df[df.index != datetime.time(0, 0, 0)] # remove no times
data = df[df.index != datetime.datetime(2022,4,22,0, 0, 0)] # remove no times

In [ ]:
data

In [ ]:
data.dtypes

In [ ]:
data.index

In [ ]:
data.groupby(pd.Grouper(freq='5Min')).count().plot(kind='bar')

In [ ]:
d = data.drop(['Category', 'Category Position',
        'Gender', 'Gender Position'], axis=1)


In [ ]:
d['Occurrences'] = 1
d.drop(['Overall Position'], axis=1, inplace=True)

In [ ]:
d.groupby(pd.Grouper(freq='20Min')).count().plot(kind='bar')

In [ ]:
res = d.groupby(pd.Grouper(freq='20Min')).count()

In [ ]:
res

In [ ]:
d_clean = d.iloc[:-11 , :] # last value is way out

In [ ]:
d_clean.groupby(pd.Grouper(freq='5Min')).count().plot(kind='bar')

In [ ]:
res_clean = d_clean.groupby(pd.Grouper(freq='5Min')).count()
res_clean.describe()

In [ ]:
print(res_clean.to_markdown())

In [ ]:
df_1= pd.read_excel (marathon_results)

In [ ]:
df_1.drop(['Bib', 'Name','Team / Club', 'Avg speed', 'Avg pace', 'Status', 'Split - 10K - Duration', 'Split - 10K - Cumulative time',
       'Split - 10K - Rank', 'Split - Halfway - Duration',
       'Split - Halfway - Cumulative time', 'Split - Halfway - Rank',
       'Split - 30K - Duration', 'Split - 30K - Cumulative time',
       'Split - 30K - Rank', 'Split - Finish - Duration',
       'Split - Finish - Cumulative time'], axis=1, inplace=True)

In [ ]:
df_1

In [ ]:
display(df_1.dtypes)

In [ ]:
print("Type of Names Column : ", type(df_1.iloc[:, 0]))

In [ ]:
print("Type of Names Column Element : ", type(df_1.iloc[:, 1][2]))

In [ ]:
print(f"time/date: {df_1.iloc[1][1]}")

In [ ]:
#df_1["Chiptime"] = pd.to_datetime(df_1["Chiptime"])
df_1['seconds'] = [(val.hour*60+val.minute)*60 + val.second for val in df_1['Chiptime']]

In [ ]:
df_1['seconds']

In [ ]:
df_1_clean = df_1[df_1['Chiptime'] != datetime.time(0, 0, 0)] # remove no times

In [ ]:
df_1_clean

In [ ]:
#seconds_in_2_hrs = 60*60*2
#df_1_clean['sec_2hr'] = df_1_clean['seconds'] - seconds_in_2_hrs

In [ ]:
#df_1_clean

In [ ]:
#df_1_clean['add_secs'] = df_1_clean['seconds'] - df_1_clean['seconds'][0]

In [ ]:
#df_1_clean

In [ ]:
#df_1_clean['add_secs'].describe()

In [ ]:
#mean_add = df_1_clean['add_secs'].mean()

In [ ]:
#mean_add

In [ ]:
#mean_stddev = df_1_clean['seconds'].std()
#print(f'mean HH:MM:SS {time.strftime("%H:%M:%S", time.gmtime(mean_stddev))}')

In [ ]:
#mean_secs = df_1_clean['seconds'].mean()
#print(f'mean = {time.strftime("%H:%M:%S", time.gmtime(mean_secs))}')

In [ ]:
#stddev_secs = df_1_clean['seconds'].std()
#print(f'std dev = {time.strftime("%H:%M:%S", time.gmtime(stddev_secs))}')

In [ ]:
print(df_1_clean['Category'].unique())

## MV45

In [ ]:
df_mv45 = df_1_clean.loc[df_1_clean['Category'] == 'MV45']
df_mv45

In [ ]:
n,bins,patches = plt.hist(df_mv45['seconds'], edgecolor="red", bins=50)

In [ ]:
bins

In [ ]:
mv45_results = ot.Sample([[p] for p in df_mv45['seconds']])
mv45_distribution = ot.GeneralizedExtremeValueFactory().buildAsGeneralizedExtremeValue(mv45_results)
print(mv45_distribution)

In [ ]:
graph = mv45_distribution.drawPDF()
graph.add(ot.HistogramFactory().build(mv45_results).drawPDF())
graph.setColors(["black", "red"])
graph.setLegends(["GEV fitting", "histogram"])
graph.setLegendPosition("topright")

view = viewer.View(graph)
axes = view.getAxes()
#_ = axes[0].set_xlim(-20.0, 20.0)

In [ ]:
print(mv45_distribution.getActualDistribution())

### Weibull distribution

In [ ]:
mean_45 = mv45_distribution.getMean()
sigma_45 = mv45_distribution.getSigma() # location
mu_45 = mv45_distribution.getMu() # scale
xi_45 = mv45_distribution.getXi() # shape
std_dev_45 = mv45_distribution.getStandardDeviation()

In [ ]:
print(f'mean MV45 = {mean_45}')
print(f'std dev MV45 = {std_dev_45}')
print(f'location MV45 = {sigma_45}')
print(f'scale MV45 = {mu_45}')
print(f'shape MV45 = {xi_45}')

In [ ]:
mv45_std_dev_time = mean_45 - std_dev_45
print('Weibull distribution')
print(f'mean MV45 = {time.strftime("%H:%M:%S", time.gmtime(int(mean_45[0])))}')
print(f'std dev MV45 = {time.strftime("%H:%M:%S", time.gmtime(int(std_dev_45[0])))}')
print(f'time at 1 std dev MV45 = {time.strftime("%H:%M:%S", time.gmtime(int(mv45_std_dev_time[0])))}')

###  Gumbel distribution (with shape ~0 this could be a Gumbel)

In [ ]:
gumbel_var = ((np.pi**2)/6)*sigma_45**2 # see wiki for variance of a gumbel plot. Here 2305.37 is scale parameter
gumbel_stddev = np.sqrt(gumbel_var)

In [ ]:
gumbel_mean = mu_45 + sigma_45*sympy.EulerGamma.evalf() # see wiki on mean calculation of Gumbel distribution

In [ ]:
print(f'mean MV45 = {time.strftime("%H:%M:%S", time.gmtime(int(gumbel_mean)))}')
print(f'var MV45 = {time.strftime("%H:%M:%S", time.gmtime(gumbel_var))}')
print(f'std dev MV45 = {time.strftime("%H:%M:%S", time.gmtime(gumbel_stddev))}')

In [ ]:
sympy.EulerGamma.evalf()

In [ ]:
g_std_dev_1_time = gumbel_mean - gumbel_stddev
print('Gumbel distribution')
print(f'mean MV45 = {time.strftime("%H:%M:%S", time.gmtime(int(gumbel_mean)))}')
print(f'std dev MV45 = {time.strftime("%H:%M:%S", time.gmtime(int(gumbel_stddev)))}')
print(f'time at 1 std dev MV45 = {time.strftime("%H:%M:%S", time.gmtime(int(g_std_dev_1_time)))}')

### As a normal distribution

In [ ]:
mean_secs_mv45 = df_mv45['seconds'].mean()
stddev_secs_mv45 = df_mv45['seconds'].std()
std_dev_1_time = mean_secs_mv45 - stddev_secs_mv45

In [ ]:
print('Normal distribution')
print(f'mean MV 45 = {time.strftime("%H:%M:%S", time.gmtime(mean_secs_mv45))}')
print(f'std dev MV45 = {time.strftime("%H:%M:%S", time.gmtime(stddev_secs_mv45))}')
print(f'1 std dev time MV45 = {time.strftime("%H:%M:%S", time.gmtime(std_dev_1_time))}')

## Males

In [ ]:
df_male = df_1_clean.loc[df_1_clean['Gender'] == 'm']
#df_male = df_1_clean.loc[df_1_clean['Category'].str.contains('M', na=False)]
df_male

### Remove the last result as it is 5 hours after the next last result!

In [ ]:
df_male = df_male[:-1]

### get distribution and type

In [ ]:
male_results = ot.Sample([[p] for p in df_male['seconds']])
male_distribution = ot.GeneralizedExtremeValueFactory().buildAsGeneralizedExtremeValue(male_results)
print(male_distribution)
print(male_distribution.getActualDistribution())

In [ ]:
graph = male_distribution.drawPDF()
graph.add(ot.HistogramFactory().build(male_results).drawPDF())
graph.setColors(["black", "red"])
graph.setLegends(["GEV fitting", "histogram"])
graph.setLegendPosition("topright")

view = viewer.View(graph)
axes = view.getAxes()
#_ = axes[0].set_xlim(-20.0, 20.0)

In [ ]:
mean_male = male_distribution.getMean()
sigma_male = male_distribution.getSigma() # location
mu_male = male_distribution.getMu() # scale
xi_male = male_distribution.getXi() # shape
std_dev_male = male_distribution.getStandardDeviation()

In [ ]:
male_std_dev_time = mean_male - std_dev_male
print(male_distribution.getActualDistribution())
print(f'mean male = {time.strftime("%H:%M:%S", time.gmtime(int(mean_male[0])))}')
print(f'std dev male = {time.strftime("%H:%M:%S", time.gmtime(int(std_dev_male[0])))}')
print(f'time at 1 std dev male = {time.strftime("%H:%M:%S", time.gmtime(int(male_std_dev_time[0])))}')

## Female FV45

In [ ]:
df_fv45 = df_1_clean.loc[df_1_clean['Category'] == 'FV45']
df_fv45

In [ ]:
fv45_results = ot.Sample([[p] for p in df_fv45['seconds']])
fv45_distribution = ot.GeneralizedExtremeValueFactory().buildAsGeneralizedExtremeValue(fv45_results)
print(fv45_distribution)
print(fv45_distribution.getActualDistribution())

In [ ]:
graph = fv45_distribution.drawPDF()
graph.add(ot.HistogramFactory().build(fv45_results).drawPDF())
graph.setColors(["black", "red"])
graph.setLegends(["GEV fitting", "histogram"])
graph.setLegendPosition("topright")

view = viewer.View(graph)
axes = view.getAxes()
#_ = axes[0].set_xlim(-20.0, 20.0)

In [ ]:
mean_fv45 = fv45_distribution.getMean()
sigma_fv45 = fv45_distribution.getSigma() # location
mu_fv45 = fv45_distribution.getMu() # scale
xi_fv45 = fv45_distribution.getXi() # shape
std_dev_fv45 = fv45_distribution.getStandardDeviation()

In [ ]:
fv45_std_dev_time = mean_fv45 - std_dev_fv45
print(fv45_distribution.getActualDistribution())
print(f'mean male = {time.strftime("%H:%M:%S", time.gmtime(int(mean_fv45[0])))}')
print(f'std dev male = {time.strftime("%H:%M:%S", time.gmtime(int(std_dev_fv45[0])))}')
print(f'time at 1 std dev fv45 = {time.strftime("%H:%M:%S", time.gmtime(int(fv45_std_dev_time[0])))}')